<a href="https://colab.research.google.com/github/vlordier/colabs/blob/main/Bot_adversarial_dialogue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ParlAI : bot_adversarial_dialogue model

#### Before we start using ParlAI, we make sure that running this session with a GPU attached:

In [ ]:
!nvidia-smi

Mon Jan 10 22:07:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installing parlai

We need to install ParlAI and all the dependencies.

In [ ]:
!pip install -q parlai
!pip install -q subword_nmt # extra requirement we need for this tutorial

     |████████████████████████████████| 1.5 MB 5.4 MB/s 
     |████████████████████████████████| 124 kB 49.1 MB/s 
     |████████████████████████████████| 208 kB 47.2 MB/s 
     |████████████████████████████████| 235 kB 43.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.7 MB 37.2 MB/s 
     |████████████████████████████████| 145 kB 48.7 MB/s 
     |████████████████████████████████| 8.5 MB 33.4 MB/s 
     |████████████████████████████████| 74 kB 3.0 MB/s 
     |████████████████████████████████| 131 kB 45.3 MB/s 
     |████████████████████████████████| 547 kB 46.0 MB/s 
     |████████████████████████████████| 6.8 MB 41.3 MB/s 
     |████████████████████████████████| 306 kB 49.9 MB/s 
     |████████████████████████████████| 170 kB 47.6 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |██████████████████

### Data
in ParlAI directory, the Bot-Adversarial Dialogue task at parlai/tasks/bot_adversarial_dialogue. To view the data, we run:

In [ ]:
!parlai display_data -t bot_adversarial_dialogue --bad-include-persona True

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
22:08:07 | Opt:
22:08:07 |     allow_missing_init_opts: False
22:08:07 |     bad_include_persona: True
22:08:07 |     bad_num_turns: -1
22:08:07 |     bad_safety_mix: all
22:08:07 |     bad_speaker_to_eval: all
22:08:07 |     batchsize: 1
22:08:07 |     datapath: /usr/local/lib/python3.7/dist-packages/data
22:08:07 |     datatype: train:ordered
22:08:07 |     dict_class: None
22:08:07 |     display_add_fields: 
22:08:07 |     download_path: None
22:08:07 |     dynamic_batching: None
22:08:07 |     hide_labels: False
22:08:07 |     ignore_agent_reply: True
22:08:07 |     image_cropsize: 224
22:08:07 |     image_mode: raw
22:08:07 |     image_size: 256
22:08:07 |     init_model: None
22:08:07 |     init_opt: None
22:08:07 |     is_debug:

To view the data used for the adversarial fixed test set, we can run:

In [ ]:
!parlai display_data -t bot_adversarial_dialogue:HumanSafetyEvaluation --bad-include-persona True

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
22:08:17 | Opt:
22:08:17 |     allow_missing_init_opts: False
22:08:17 |     bad_include_persona: True
22:08:17 |     batchsize: 1
22:08:17 |     datapath: /usr/local/lib/python3.7/dist-packages/data
22:08:17 |     datatype: train:ordered
22:08:17 |     dict_class: None
22:08:17 |     display_add_fields: 
22:08:17 |     download_path: None
22:08:17 |     dynamic_batching: None
22:08:17 |     flatten_dialogue: True
22:08:17 |     hide_labels: False
22:08:17 |     ignore_agent_reply: True
22:08:17 |     image_cropsize: 224
22:08:17 |     image_mode: raw
22:08:17 |     image_size: 256
22:08:17 |     init_model: None
22:08:17 |     init_opt: None
22:08:17 |     is_debug: False
22:08:17 |     loglevel: info
22:08:17 |     max_display_len: 1

We can show the validation set for the sensitive topics detection. The data can be viewed with the following command:

In [ ]:
!parlai display_data -t sensitive_topics_evaluation -dt valid

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
22:08:23 | Opt:
22:08:23 |     allow_missing_init_opts: False
22:08:23 |     batchsize: 1
22:08:23 |     datapath: /usr/local/lib/python3.7/dist-packages/data
22:08:23 |     datatype: valid
22:08:23 |     dict_class: None
22:08:23 |     display_add_fields: 
22:08:23 |     download_path: None
22:08:23 |     dynamic_batching: None
22:08:23 |     hide_labels: False
22:08:23 |     ignore_agent_reply: True
22:08:23 |     image_cropsize: 224
22:08:23 |     image_mode: raw
22:08:23 |     image_size: 256
22:08:23 |     init_model: None
22:08:23 |     init_opt: None
22:08:23 |     is_debug: False
22:08:23 |     loglevel: info
22:08:23 |     max_display_len: 1000
22:08:23 |     model: None
22:08:23 |     model_file: None
22:08:23 |     multitask

See below for a classifier trained to predict these topics.

### **Models**
##### **BAD classifier**
A classifier trained on the new Bot-Adversarial Dialogue (BAD) task (as well as other existing safety tasks) can be found at zoo:bot_adversarial_dialogue/multi_turn/model.

This model can be downloaded and evaluated on the BAD task test set with the following command:

In [ ]:
!parlai eval_model -t bot_adversarial_dialogue:bad_num_turns=4 -dt test -mf zoo:bot_adversarial_dialogue/multi_turn/model -bs 128

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
22:08:28 | building data: /usr/local/lib/python3.7/dist-packages/data/models/bot_adversarial_dialogue/multi_turn/models_v1.tar.gz
22:08:28 | Downloading http://parl.ai/downloads/_models/bot_adversarial_dialogue/multi_turn/models_v1.tar.gz to /usr/local/lib/python3.7/dist-packages/data/models/bot_adversarial_dialogue/multi_turn/models_v1.tar.gz
22:12:28 | Overriding opt["task"] to bot_adversarial_dialogue:bad_num_turns=4 (previously: dialogue_safety:WikiToxicComments,dialogue_safety:standard,dialogue_safety:adversarial,internal:convai2_review,internal:safeways:safeways_mix=unsafe:safeways_version=2,internal:safeways:safeways_mix=safe:safeways_version=2)
22:12:28 | Overriding opt["datatype"] to test (previously: train)
22:12:28 | Overrid

### **Looking at model predictions**
We have trained the model, interactively. We might want to inspect how the model reacts with a fixed set of inputs. Let's use that model we just trained!

In [ ]:
!python -m parlai.scripts.display_model --task bot_adversarial_dialogue:bad_num_turns=4 --model-file zoo:bot_adversarial_dialogue/multi_turn/model

22:17:26 | Overriding opt["task"] to bot_adversarial_dialogue:bad_num_turns=4 (previously: dialogue_safety:WikiToxicComments,dialogue_safety:standard,dialogue_safety:adversarial,internal:convai2_review,internal:safeways:safeways_mix=unsafe:safeways_version=2,internal:safeways:safeways_mix=safe:safeways_version=2)
22:17:26 | Overriding opt["model_file"] to /usr/local/lib/python3.7/dist-packages/data/models/bot_adversarial_dialogue/multi_turn/model (previously: /checkpoint/jingxu23/safeways/eval_safety/adv_clf/finetunesafetyv2_adv_0_v2_again/3858/model)
22:17:26 | Using CUDA
22:17:26 | loading dictionary from /usr/local/lib/python3.7/dist-packages/data/models/bot_adversarial_dialogue/multi_turn/model.dict
22:17:26 | num words = 8008
22:17:26 | Are you sure you want to lower case your BPE dictionary?
22:17:37 | Loading existing model parameters from /usr/local/lib/python3.7/dist-packages/data/models/bot_adversarial_dialogue/multi_turn/model
22:19:10 | Total parameters: 311,037,954 (311,03


##**Interaction with the chatbot:**

Interact with the bot_adversarial_dialogue model to see its classifications of the inputs in real time. So, here some example i'v tried from BAD model using the **single-turn** task (detection of offensive language in one utterance, with
no dialogue context or conversational history):

In [ ]:
!parlai interactive -mf zoo:bot_adversarial_dialogue/multi_turn/model -m transformer/classifier --print-scores True --single-turn True

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
22:23:54 | Overriding opt["model_file"] to /usr/local/lib/python3.7/dist-packages/data/models/bot_adversarial_dialogue/multi_turn/model (previously: /checkpoint/jingxu23/safeways/eval_safety/adv_clf/finetunesafetyv2_adv_0_v2_again/3858/model)
22:23:54 | Overriding opt["print_scores"] to True (previously: False)
22:23:54 | Using CUDA
22:23:54 | loading dictionary from /usr/local/lib/python3.7/dist-packages/data/models/bot_adversarial_dialogue/multi_turn/model.dict
22:23:54 | num words = 8008
22:23:54 | Are you sure you want to lower case your BPE dictionary?
22:23:54 | TransformerClassifier: full interactive mode on.
22:24:05 | Loading existing model parameters from /usr/local/lib/python3.7/dist-packages/data/models/bot_adversarial_dial

### **Multiturn task**
on this task we focus on offensive utterances within the context of two-person dialogues.

Evaluate a BERT-based model (using segments to distinguish the context from the utterance to be classified) on the test set of the multi-turn adversarial task

In [ ]:
!pip install pytorch-pretrained-bert #library need it to use the BERT-based model from pytorch

     |████████████████████████████████| 123 kB 5.3 MB/s 


In [ ]:
!parlai eval_model -t dialogue_safety:multiturn -dt test -mf zoo:dialogue_safety/multi_turn/model --split-lines True -bs 40

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
23:33:35 | Overriding opt["task"] to dialogue_safety:multiturn (previously: internal:safety:wikiToxicComments,internal:safety:boringConvAI2Review,internal:safety:adversarialConvAI2Review,internal:safety:multiturnConvAI2)
23:33:35 | Overriding opt["datatype"] to test (previously: train)
23:33:35 | Overriding opt["model_file"] to /usr/local/lib/python3.7/dist-packages/data/models/dialogue_safety/multi_turn/model (previously: /checkpoint/edinan/20190903/multiturn_rerun2/single-turn=False_sep-last-utt=True_multitask-weights=0.5,0.1,0.1,0.3_lr=5e-05_lr-scheduler-patience=3_lr-scheduler-decay=0.9_warmupupdates=2000/model)
[downloading BERT models: /usr/local/lib/python3.7/dist-packages/data/models/bert_models]
23:33:35 | Downloading https://

In [ ]:
!parlai interactive -t dialogue_safety:multiturn -dt test -mf zoo:dialogue_safety/multi_turn/model --split-lines True -bs 40

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
23:46:40 | Overriding opt["task"] to dialogue_safety:multiturn (previously: internal:safety:wikiToxicComments,internal:safety:boringConvAI2Review,internal:safety:adversarialConvAI2Review,internal:safety:multiturnConvAI2)
23:46:40 | Overriding opt["datatype"] to test (previously: train)
23:46:40 | Overriding opt["model_file"] to /usr/local/lib/python3.7/dist-packages/data/models/dialogue_safety/multi_turn/model (previously: /checkpoint/edinan/20190903/multiturn_rerun2/single-turn=False_sep-last-utt=True_multitask-weights=0.5,0.1,0.1,0.3_lr=5e-05_lr-scheduler-patience=3_lr-scheduler-decay=0.9_warmupupdates=2000/model)
23:46:40 | Using CUDA
23:46:40 | loading dictionary from /usr/local/lib/python3.7/dist-packages/data/models/dialogue_safe